In [ ]:
from copulas.multivariate import GaussianMultivariate
import warnings
warnings.filterwarnings('ignore')

# ローリングウィンドウでのコピュラフィッティング
window_size = 60  # 約3ヶ月
anomaly_scores = []
fitted_rhos = []
fitted_dates = []

print(f"ローリング分析開始（ウィンドウサイズ: {window_size}日）...")

for i in range(window_size, len(returns)):
    if i % 100 == 0:
        print(f"進捗: {i-window_size+1}/{len(returns)-window_size}")
    
    # ウィンドウデータ取得
    window_data = returns.iloc[i-window_size:i]
    
    # 一様分布への変換
    u_window = to_uniform_marginals(window_data['Nikkei'])
    v_window = to_uniform_marginals(window_data['SP500'])
    
    # ガウシアンコピュラをフィッティング
    copula_data = pd.DataFrame({'u': u_window, 'v': v_window})
    
    try:
        # copulasライブラリを使用
        gaussian_copula = GaussianMultivariate()
        gaussian_copula.fit(copula_data)
        
        # 今日のデータポイント
        today_return_nikkei = returns['Nikkei'].iloc[i]
        today_return_sp500 = returns['SP500'].iloc[i]
        
        # 今日のデータを一様分布値に変換
        extended_nikkei = list(window_data['Nikkei']) + [today_return_nikkei]
        extended_sp500 = list(window_data['SP500']) + [today_return_sp500]
        
        u_today = (rankdata(extended_nikkei))[-1] / (len(extended_nikkei) + 1)
        v_today = (rankdata(extended_sp500))[-1] / (len(extended_sp500) + 1)
        
        # コピュラ尤度計算（異常スコア）
        likelihood = gaussian_copula.pdf(pd.DataFrame({'u': [u_today], 'v': [v_today]}))
        anomaly_score = -np.log(likelihood.iloc[0] + 1e-10)  # 負の対数尤度
        
        anomaly_scores.append(anomaly_score)
        fitted_rhos.append(gaussian_copula.covariance[0, 1])  # 相関パラメータ
        fitted_dates.append(returns.index[i])
        
    except Exception as e:
        # エラーの場合はNaNを追加
        anomaly_scores.append(np.nan)
        fitted_rhos.append(np.nan)
        fitted_dates.append(returns.index[i])

# 結果をDataFrameに
results_df = pd.DataFrame({
    'Date': fitted_dates,
    'Nikkei_Return': returns['Nikkei'].iloc[window_size:].values,
    'SP500_Return': returns['SP500'].iloc[window_size:].values,
    'Anomaly_Score': anomaly_scores,
    'Fitted_Rho': fitted_rhos
})

# NaNを除去
results_df = results_df.dropna()

print(f"分析完了！有効なデータポイント: {len(results_df)}個")
print(f"異常スコア範囲: {results_df['Anomaly_Score'].min():.3f} ～ {results_df['Anomaly_Score'].max():.3f}")

In [ ]:
# 異常の定義: スコアの上位5%
threshold = results_df['Anomaly_Score'].quantile(0.95)
anomalies = results_df['Anomaly_Score'] > threshold

# 異常検知結果
anomaly_dates = results_df[anomalies]['Date']
anomaly_scores_high = results_df[anomalies]['Anomaly_Score']

print(f"異常検知閾値: {threshold:.3f}")
print(f"検知された異常の数: {len(anomaly_dates)}")
print(f"異常検知率: {len(anomaly_dates)/len(results_df)*100:.2f}%")

# 主要な異常イベントの確認
major_anomalies = results_df[anomalies].nlargest(10, 'Anomaly_Score')
print("\n=== 上位10の異常イベント ===")
for idx, row in major_anomalies.iterrows():
    print(f"{row['Date'].strftime('%Y-%m-%d')}: "
          f"Score={row['Anomaly_Score']:.3f}, "
          f"Nikkei={row['Nikkei_Return']:.3%}, "
          f"SP500={row['SP500_Return']:.3%}")

# 実際の金融イベントとの対応
crisis_periods = {
    'COVID-19': ('2020-02-20', '2020-04-01'),
    'Russia-Ukraine': ('2022-02-24', '2022-03-31'),
    'Trade War': ('2018-10-01', '2018-12-31'),
    'Brexit': ('2016-06-23', '2016-07-31'),
}

print("\n=== 主要金融危機での異常検知 ===")
for crisis_name, (start, end) in crisis_periods.items():
    crisis_mask = (results_df['Date'] >= start) & (results_df['Date'] <= end)
    crisis_data = results_df[crisis_mask]
    crisis_anomalies = crisis_data[crisis_data['Anomaly_Score'] > threshold]
    
    if len(crisis_data) > 0:
        detection_rate = len(crisis_anomalies) / len(crisis_data) * 100
        print(f"{crisis_name}: {len(crisis_anomalies)}/{len(crisis_data)}件 ({detection_rate:.1f}%)")
        
        if len(crisis_anomalies) > 0:
            max_score = crisis_anomalies['Anomaly_Score'].max()
            max_date = crisis_anomalies.loc[crisis_anomalies['Anomaly_Score'].idxmax(), 'Date']
            print(f"  最大スコア: {max_score:.3f} ({max_date.strftime('%Y-%m-%d')})")

# 異常検知の統計分析
normal_data = results_df[~anomalies]
anomaly_data = results_df[anomalies]

print("\n=== 統計分析 ===")
print("通常時:")
print(f"  平均日経収益率: {normal_data['Nikkei_Return'].mean():.4f}")
print(f"  平均S&P500収益率: {normal_data['SP500_Return'].mean():.4f}")
print(f"  平均相関: {normal_data['Fitted_Rho'].mean():.3f}")

print("異常時:")
print(f"  平均日経収益率: {anomaly_data['Nikkei_Return'].mean():.4f}")
print(f"  平均S&P500収益率: {anomaly_data['SP500_Return'].mean():.4f}")
print(f"  平均相関: {anomaly_data['Fitted_Rho'].mean():.3f}")